In [65]:
import sqlite3 as sql 
import numpy as np
import pandas as pd

In [66]:
db = "plankermira/out/Nordwandschacht.sql"
CAVENAME = "Nordwandschacht"
conn = sql.connect(':memory:')
conn.executescript(open(db).read())

In [67]:
c = conn.cursor()

c.execute("SELECT ct.TITLE, ct.TOPO_DATE, su2.NAME, su1.NAME, stn1.NAME, su2.NAME, su1.NAME, stn2.NAME, sh.LENGTH, sh.BEARING, sh.GRADIENT, sf.FLAG \
          from STATION as stn1, STATION as STN2, SHOT as sh, SURVEY as su1, SURVEY as su2, CENTRELINE as ct\
          LEFT OUTER JOIN SHOT_FLAG sf \
          ON sh.ID == sf.SHOT_ID \
          where stn1.ID == sh.FROM_ID and stn2.ID == sh.TO_ID and stn2.NAME != '-' and stn2.NAME != '.' \
          and stn1.SURVEY_ID == su1.ID and su1.PARENT_ID == su2.ID and sh.CENTRELINE_ID == ct.ID;")

In [68]:
shot_data = pd.DataFrame(c.fetchall())

In [69]:
series_titles= shot_data[0].unique()
series_dates = shot_data[1].unique()

In [70]:
topoteams = []
dates = []
for series_date in series_dates:
    topoteam = ""
    sep = " "
    date = sep.join(series_date.split("-")[::-1])
    monthfirst = date[1] + date[0] + date[2]

    c.execute(f"SELECT pp.NAME, pp.SURNAME from PERSON pp, CENTRELINE ct, TOPO tp\
              where tp.PERSON_ID == pp.ID and ct.ID == tp.CENTRELINE_ID and ct.TOPO_DATE == '{series_date}';")
    for name, surname in c.fetchall():
        topoteam+= f"{name} {surname}, "
    
    topoteams.append(topoteam[:-2])
    dates.append(monthfirst)


In [71]:
series_titles

array(['Oberer Teil der Tiefenbronner Halle',
       'Unterer Teil der Tiefenbronner Halle', 'Zugzwang Gang',
       'Eingangschächte der Nordwandschacht', 'Aglarond', 'Vesuv-Dom',
       'Shawshank-Redemption-Schluf'], dtype=object)

In [72]:
duplicate_shot = "#|L#"
survey_name = "SURVEY NAME: {survey_name}"
survey_date = "SURVEY DATE: {survey date}"
survey_team = "SURVEY TEAM: \n{survey team}"
centreline_header = """DECLINATION:    0.00  FORMAT: DMMDLRUDLDAN  CORRECTIONS:  0.00 0.00 0.00

        FROM           TO   LENGTH  BEARING      INC     LEFT       UP     DOWN    RIGHT   FLAGS  COMMENTS"""

In [73]:
feet = 3.2808399

In [74]:
shot_stub = """{: >12}{: >13}{: >9}{: >9}{: >9}     0.00     0.00     0.00     0.00  {}\n"""

In [75]:
compass_file = ""

for serie, title in zip(series_dates, series_titles):

    topoteam = ""
    sep = " "
    date = serie.split("-")[::-1]
    monthfirst = f"{date[1]} {date[0]} {date[2]}"
    

    c.execute(f"SELECT pp.NAME, pp.SURNAME from PERSON pp, CENTRELINE ct, TOPO tp\
              where tp.PERSON_ID == pp.ID and ct.ID == tp.CENTRELINE_ID and ct.TOPO_DATE == '{serie}';")
    
    for name, surname in c.fetchall():
        if f"{name} {surname}" not in topoteam:
            topoteam+= f"{name} {surname}, "

    centreline_stub = """{cavename}
SURVEY NAME: {formatted_name}
SURVEY DATE: {formatted_date}   COMMENT:
SURVEY TEAM:
{formatted_team}
{header}

{shots}
"""
    # extract the shot data  
    ct = shot_data[shot_data[1] == serie]

    formatted_shots = ""

    for index, shot in ct.iterrows():

        stn_from = shot[2] +"." + shot[3] +"." + shot[4]
        stn_to = shot[5] +"." + shot[6] +"." + shot[7]
        tape = f"{float(shot[8]) * feet:.2f}"
        comp = shot[9] 
        inc = shot[10]

        if shot[11] == "dpl":
            flag = duplicate_shot
        else:
            flag = " "

        formatted_shots+= shot_stub.format(stn_from, stn_to, tape, comp, inc, flag)


    formatted_centreline = centreline_stub.format(cavename =CAVENAME,
                                                  formatted_name = title, 
                                                  formatted_date = monthfirst,
                                                  formatted_team = topoteam[:-2],
                                                  header = centreline_header,
                                                  shots = formatted_shots)
    
    compass_file += formatted_centreline

In [76]:
with open("plankermira/out/Nordwandschacht.DAT", "w") as f:
    f.write(compass_file)
    f.close()